# RPN

Afegim a la U-Net una branca nova, la branca de *region proposal network (RPN)*.  Introduida per primer cop per la *faster rcnn* duu a terme dues tasques alhora, per una part refina tot un conjunt de <a hfre="https://www.termcat.cat/ca/cercaterm/bounding%20box?type=basic">envolupants </a> i per l'altra indica quina és la probabilitat que cada un d'ells contengui un objecte.

<img style="width:75%" src="https://www.researchgate.net/publication/333048961/figure/fig1/AS:758094162296847@1557755141554/The-framework-of-Faster-R-CNN-RPN-region-proposal-network-RoI-region-of-interest-FC.ppm" />

### Importam llibreries

In [ ]:
import os
import json
import colorsys
import random
from datetime import datetime
import logging
from logging.handlers import TimedRotatingFileHandler
import sys

import cv2 
import skimage
import skimage.io
import skimage.color
import skimage.transform
import numpy as np
import pandas as pd
import imgaug as ia
import imgaug.augmenters as iaa
from numpy.random import seed
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from matplotlib import patches,  lines

from tensorflow.keras import backend as K
import tensorflow.keras.layers as keras_layer
import tensorflow as tf
from tensorflow.keras import utils as KU
import keras


# Llibraries pròpies
from u_rpn.data import unet as u_data
from u_rpn.data import rpn as rpn_data
from u_rpn.data import datasets as rpn_datasets
from u_rpn import model as u_model
from u_rpn import configurations as u_configs
from u_rpn.common import data as common_data
from u_rpn import layers as own_layers
from u_rpn.losses import bce
from u_rpn import losses as rpn_losses
from u_rpn.common import utils as rpn_utils
from u_rpn.common import metrics as rpn_metrics


seed(1)

In [ ]:
# ============================================
# Optimisation Flags - Do not remove
# ============================================

os.environ['CUDA_CACHE_DISABLE'] = '0'

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT'] = '1'

os.environ['TF_USE_CUDNN_BATCHNORM_SPATIAL_PERSISTENT'] = '1'

os.environ['TF_ADJUST_HUE_FUSED'] = '1'
os.environ['TF_ADJUST_SATURATION_FUSED'] = '1'
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

os.environ['TF_SYNC_ON_FINISH'] = '0'
os.environ['TF_AUTOTUNE_THRESHOLD'] = '2'
os.environ['TF_DISABLE_NVTX_RANGES'] = '1'

# =================================================

In [ ]:
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def draw_bboxes(img, bboxes, thickness=3):
    img = np.copy(img.astype(np.uint8))
    colors = random_colors(len(bboxes))

    for bbox, color in zip(bboxes, colors):
        color = np.array(color) * 255
        img = cv2.rectangle(img, (bbox[1], bbox[0]), (bbox[3], bbox[2]), color, thickness)
    
    return img

def make_masks(mask, slice_mask):
    if isinstance(slice_mask, list):
        if len(slice_mask) != 2:
            raise ValueError
        
        mask = np.sum(mask[:,:, slice_mask[0]:slice_mask[1]], axis=-1)
    elif isinstance(slice_mask, list):
        mask = mask[:, :, slice_mask]
    else:
        raise ValueError
    
    return mask

### Configuració

Primerament cream un classe configuració per l'execusió i entrenament de la xarxa. En aquesta classe deixam els valors per defecte exceptuant els casos del nombre de classes, la mida de les ancores, les pases per època i el llindar mínim de confiança.

In [ ]:
MULTI_CLASS = False
PYRAMID = False
TRANSFER = False

In [ ]:
config = u_configs.CellConfig()

# Number of classes (including background)
if MULTI_CLASS:
    NUM_CLASSES = 1 + 3  # Background + 3 classes
else:
    NUM_CLASSES = 1 + 1  # Background + 3 classes
        
config.IMAGE_SHAPE = np.array([128,128, 3])
config.BATCH_SIZE = 2
config.DO_MERGE_BRANCH = False
config.DO_MASK_CLASS = False

print(config)

## Entrenament

Per realitzar l'entrenament primerament cream dos generadors d'imatges. Els generadors en el cas de la *RPN* es creen en dos temps. Primerament cream objectes **Dataset**.

![SegmentLocal](https://imgs.xkcd.com/comics/ai_methodology.png "segment")

### Dataset

Definim un objecte Dataset. Anàlogament a la configuració, ja definida, és basa en herència de classes abstractes definides a les llibreries. Un detall important és que en el cas de la RPN les dades es formen a partir dels envolupants, enlloc de l'inrevés.

In [ ]:
# Validation dataset
dataset_val = rpn_datasets.ErithocytesDataset([("cell", 1, "cell")], "bboxes.json")
dataset_val.load_cell("./in/eritocitos_augmented/", rpn_datasets.Subset.VALIDATION)
dataset_val.prepare()

val_generator = rpn_data.DataGenerator(2, dataset_val, config, shuffle=False)
anchors = val_generator.anchors.tolist()

if JUMBO:
    dataset = rpn_datasets.ErithocytesPreDataset(
        "./in/jumbo2_mini/train", "data.json", divisor=1
    )
    
else:
    dataset = rpn_datasets.ErithocytesPreDataset("./in/pre_calculate/train", "data.json", divisor=255)

dataset.prepare()

generator = rpn_data.DataGenerator(
    50, 
    dataset, 
    pre_calculated=True,
    config=config,
    phantom_output=True, 
    shuffle=False,
    size_anchors=dataset.anchors
)

In [ ]:
for data, _ in generator:
    break
for d in data:
    print(d.shape)

## Construim el model

### Callback

In [ ]:
class ImageHistory(keras.callbacks.Callback):

    def __init__(self, data, tensor_board_dir: str, draw_interval: int = 1,  *args, **kwargs):
        self.tensor_board_dir = tensor_board_dir
        self.draw_interval = draw_interval
        self.data = data
        self._last_step = 0
        
        super(ImageHistory, self).__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs={}):
        for item in self.data:
            y_pred = self.model.predict(item)
            self.saveToTensorBoard(y_pred[0][0,:,:,:1].reshape(1, 512, 512, 1), y_pred[0][0,:,:,1:2].reshape(1, 512, 512, 1))
                
    def saveToTensorBoard(self, img1, img2):
        writer = tf.summary.create_file_writer(self.tensor_board_dir)
        
        with writer.as_default():
            tf.summary.image("TD 1", img1, step=self._last_step)
            tf.summary.image("TD 2", img2, step=self._last_step)
            self._last_step += 1

### Model


In [ ]:
tf.no_gradient("CombinedNonMaxSuppression")

In [ ]:
urpn = u_model.u_rpn.URPN(
        config=config,
        mode=u_model.keras_rpn.NeuralMode.TRAIN,
        anchors=anchors,
        input_size=(128, 128, 3),
        decoder_output=u_model.decoder.SuperDecoderOutput,
        input_sd=(128, 128),
        cell_shape=(9, 9),
        filters=32,
        decoder_output_size=3
    )

urpn.compile(optimizer=tf.keras.optimizers.Adadelta(config.LEARNING_RATE),
             run_eagerly=True)



In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./check/checkpoint.h5', save_best_only=True)
]

urpn.internal_model.fit(
    generator,
    epochs=1000,
    steps_per_epoch=2,
    callbacks=callbacks
)

## Inferència


![SegmentLocal](https://c.tenor.com/kWejy2kDcTwAAAAC/office.gif "segment") 

### JUMBO

Sortida amb múltiples cel·les i multiples màscares per cel·la. Els objectes inclosos dins cada cel·la són segmentat per aquestes màscares. Similar a la idea de Wang *et al.*

In [ ]:
urpn.internal_model.load_weights("./check/checkpoint.h5", by_name=True)

In [ ]:
pred = urpn.internal_model.predict(data)

In [ ]:
for p in pred:
    break

In [ ]:
p[p < 0.5]=0

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 15]

for i in range(80):
    plt.subplot(9, 9, i+1)
    cell_x = int(i // 9)
    cell_y = int(i % 9)
    plt.imshow(p[0, :, :, cell_x, cell_y, 1])
    

### NORMAL

Per realitzar la inferència generam un nou model, amb el mode ``INFERENCE``. Una vegada creat hem de carregar els pesos des d'un fitxer, generat quan acabam l'entrenament.

In [ ]:
rpn = build_model(u_model.rpn.NeuralMode.INFERENCE, config, True)
rpn.load_weights("./check/checkpoint.h5")

In [ ]:
for i, (t, _) in enumerate(train_generator):
    break
    
if True:
    masks, cls, bboxes, msk_cls, mask_branch = rpn.internal_model.predict(t[0])
else:
    masks, _, cls, bboxes, _, _, _, msk_cls = rpn.internal_model.predict(t)

In [ ]:
print(f"Mask class - Pred {np.count_nonzero(msk_cls[0] > 0.5)} | GT {np.count_nonzero(t[4][0] > 0.5)}")
print("***"*10)

%matplotlib inline

print("Masks")

plt.figure(figsize=(25, 25))
for i in range(36):
    plt.subplot(6, 6, i + 1)
    mask = masks[0,:,:,i]
    mask[mask < 0.6] = 0 
    plt.imshow(mask, vmin=0, vmax=1)

In [ ]:
WINDOW = [64, 0, 448, 512]
ORG_IMG = [2352, 3136, 3]
IMG_SHAPE = [512, 512, 3]

def predict_and_show(org_img, threshold):
    in_img_m_r, window, _, _, _ = rpn_utils.resize_image(org_img, min_dim = 512, max_dim=512, mode=config.IMAGE_RESIZE_MODE)
    in_img_m = val_generator.mold_image(in_img_m_r)
    
    img_batch = np.repeat(in_img_m.reshape((1, 512, 512, 3)), 5, axis=0)
    masks, cls, bboxes, msk_cls = rpn.predict(img_batch)

    # mask_pred, cls, bboxes, msk_cls = rpn.predict(img.reshape(1, 512, 512, 3))

    bboxes_deltas = bboxes[0] * config.RPN_BBOX_STD_DEV
    bboxes_deltas = val_generator.decode_deltas(bboxes_deltas)

    bboxes_filtered = bboxes_deltas[cls[0][:, 1] > threshold]
    cls =  cls[0][:, 1][cls[0][:, 1] > threshold]

    inside_the_box = ((bboxes_filtered[:, 0] > WINDOW[0] + 5) & 
                       (bboxes_filtered[:, 1] > WINDOW[1] + 5) & 
                       (bboxes_filtered[:, 2] < WINDOW[2] - 5) & 
                       (bboxes_filtered[:, 3] < WINDOW[3] - 5))
    print(len(bboxes_filtered))
    bboxes_filtered = bboxes_filtered[inside_the_box]
    cls = cls[inside_the_box]
    print(len(bboxes_filtered))
    

    bboxes_filtered = common_data.non_max_suppression_fast(bboxes_filtered, 0.3, cls)
    print(len(bboxes_filtered))
    
    res = draw_bboxes(in_img_m_r, bboxes_filtered, 1)
    
    return res

In [ ]:
org_img = skimage.io.imread("./in/bboxes_class/val/5.png")

img = predict_and_show(org_img, 0.9)

plt.figure(figsize=(15, 15))
plt.imshow(img);

In [ ]:
for l in rpn.internal_model.layers:
    print(l.name)

In [ ]:
rpn.internal_model.layers[31]

In [ ]:
from keras.models import Model

XX = rpn.internal_model.input 
YY = rpn.internal_model.layers[31].output
new_model = Model(XX, YY)

In [ ]:
in_img_m_r, window, _, _, _ = rpn_utils.resize_image(org_img, min_dim = 512, max_dim=512, mode=config.IMAGE_RESIZE_MODE)
in_img_m = val_generator.mold_image(in_img_m_r)

img_batch = np.repeat(in_img_m.reshape((1, 512, 512, 3)), 5, axis=0)
out = new_model.predict(img_batch)

In [ ]:
plt.figure(figsize=(10,10))
aux = np.sum(out, axis=-1)
plt.imshow(aux[0, :, :])

In [ ]:
gt = t[1].astype(np.float64)
plt.figure(figsize=(25, 25))
for i in range(16):
    plt.subplot(8, 4, i + 1)
    plt.imshow(out[0,:,:,i])
    
    plt.subplot(8, 4, 16 + (i + 1))
    plt.imshow(gt[0,:,:,i])

# Mètriques

In [ ]:
THRESH = 0.9
gt_g = []
p_g = []

dataset = dataset_val
generador = val_generator
resultats = []
diff_gen = 0
for idx in tqdm(dataset.image_ids):
    org_img, _, _ , gt_bbox, mask_gt = rpn_data.DataGenerator.load_image_gt(dataset, config, idx)
    img = np.copy(org_img)
    
    img = generador.mold_image(img)
    mask_pred, cls, bboxes, msk_cls = rpn.predict(img.reshape(1, 512, 512, 3))
    
    bboxes_deltas = bboxes[0] * config.RPN_BBOX_STD_DEV
    bboxes_deltas = generador.decode_deltas(bboxes_deltas)
    
    bboxes_filtered = bboxes_deltas[cls[0][:, 1] > THRESH]
    cls =  cls[0][:, 1][cls[0][:, 1] > THRESH]

    inside_the_box = ((bboxes_filtered[:, 0] > WINDOW[0] + 5) & 
                       (bboxes_filtered[:, 1] > WINDOW[1] + 5) & 
                       (bboxes_filtered[:, 2] < WINDOW[2] - 5) & 
                       (bboxes_filtered[:, 3] < WINDOW[3] - 5))
    
    bboxes_filtered = bboxes_filtered[inside_the_box]
    cls = cls[inside_the_box]
    
    bboxes_filtered = common_data.non_max_suppression_fast(bboxes_filtered, 0.3, cls)
    res = draw_bboxes(org_img, bboxes_filtered, 1)
    
    img_path = os.path.join(".", "out", "res")
    os.makedirs(img_path, exist_ok=True)
    cv2.imwrite(os.path.join(img_path, f"{idx}.png"), res)
    
    _, _, pred = rpn_metrics.relate_bbox_to_gt(bboxes_filtered, gt_bbox)

    gt_p = [1] * len(pred)

    if len(pred) < len(bboxes_filtered):
        diff = len(bboxes_filtered) - len(pred)
        pred = pred + [1] * diff
        gt_p = gt_p + [0] * diff
        
    gt_g = gt_g + gt_p
    p_g = p_g + pred
    metrics = list(rpn_metrics.basic_metrics(gt_p, pred))
    
    msk_cls[msk_cls < 0.5] = 0
    diff = np.abs(mask_gt.shape[-1] - np.count_nonzero(msk_cls))
    diff_gen += diff
    
    metrics.append(diff)
#     rpn_losses.onw_dice_coefficient()
    
    resultats.append(metrics)

diff_gen /= len(dataset.image_ids)
resultats.append(list(rpn_metrics.basic_metrics(gt_g, p_g)) + [diff_gen])
df = pd.DataFrame(resultats)
df.columns = ['Precision BB', 'Recall BB', 'F1 BB', "Diff MSK_CLS"]

# df.to_csv(os.path.join(img_path, "resultats_experiment_.csv"))
df

In [ ]:
os.path.join(img_path, "resultats_experiment_.csv")